PianoRoll


In [23]:
# !pip install pypianoroll

In [38]:
import pypianoroll
import glob
import mido
import numpy as np

In [39]:
def midi_to_pianoroll(files_path):
    n = 600
    piano_roll = []
    for file in glob.glob(files_path):
        track_count = 0
        multitrack = pypianoroll.read(file)
        pianorolls = multitrack.tracks
        multitrack_for_piano_roll = pypianoroll.Multitrack()
        for track in pianorolls:
            if(track.is_drum == False):
                multitrack_for_piano_roll.append(pypianoroll.StandardTrack(pianoroll=track.pianoroll, program=track_count))
                track_count += 1
                
        # multitrack_for_piano_roll.plot()
        track_piano_roll = multitrack_for_piano_roll.blend()
        result = [track_piano_roll[i:i+n] for i in range(0, len(track_piano_roll), n) if len(track_piano_roll[i:i+n]) == n]
        for i in result:
            i[i != 0] = 1
            piano_roll.append(i)

    return piano_roll

In [40]:
def pianoroll_to_midi(piano_rolls, file_path):
    piano_roll = np.concatenate(piano_rolls, axis=0)
    multitrack_out = pypianoroll.Multitrack()
    multitrack_out.append(pypianoroll.StandardTrack(pianoroll=piano_roll, program=0))
    multitrack_out.write(file_path)


## Example

Пример преобразования midi в piano roll с помощью функции midi_to_pianoroll, описанной выше

In [41]:
files_path = "input-sounds/genres/test/*.mid"
piano_rolls = midi_to_pianoroll(files_path)

c:\Python311\Lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Сохранение и загрузка piano_rolls(Будет нужно для обучения и генерации)

In [42]:
np.save('generated/piano_rolls/classical/piano_rolls.npy', piano_rolls)

In [45]:
# loaded_piano_rolls = np.load('generated/piano_rolls/classical/piano_rolls.npy', allow_pickle=True)
# loaded_piano_rolls = loaded_piano_rolls*127

Преобразование piano_rolls в midi файл

In [53]:
generated_piano_roll = np.load('generated/generated_piano_rolls/piano_rolls.npy', allow_pickle=True)
generated_piano_roll = generated_piano_roll*127

In [54]:
pianoroll_to_midi(generated_piano_roll, "generated/tmp/example_out.mid")

Увеличение темпа (преобразование в piano roll, почти в два раза замедляет midi)

In [49]:
files_path = "generated/tmp/*.mid"
for file, i in zip(glob.glob(files_path), range(len(glob.glob(files_path)))):
    mid = mido.MidiFile(file)

    new_tempo = mido.bpm2tempo(240)
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                msg.tempo = new_tempo

    mid.save("generated/tmp/speedup_example_out_" + str(i) + ".mid")
